In [ ]:
!pip install -q --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from itertools import chain
from time import sleep

import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from tqdm import tqdm

In [ ]:
all_results = []
as_csv = "block,form_id,responder_uri,study_entry_ids\n"

In [ ]:
SERVICE_ACCOUNT_FILE = "../../research-430307-ad3438ad46d0.json"
APPROVAL_URL = "https://app.prolific.com/submissions/complete?cc=CWYHF4HQ"
SCOPES = ["https://www.googleapis.com/auth/forms", "https://www.googleapis.com/auth/drive"]

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

forms_service = build("forms", "v1", credentials=credentials)
drive_service = build("drive", "v3", credentials=credentials)

data = pd.read_csv("../../generated-dataset-30_000/human-study/open/study_data_open.csv")

In [ ]:
BLOCK_SIZE = 20
TOTAL_NUMBER = 820  # len(data)
assert TOTAL_NUMBER % BLOCK_SIZE == 0


def create_form(block, form_data):
    # Creates the initial form
    title = f"QuAnTS Motion Question Answering #{block} (open)"
    result = (
        forms_service.forms()
        .create(
            body={
                "info": {"title": title},
            }
        )
        .execute()
    )
    form_id = result["formId"]

    list_of_actions = [
        "playing guitar",
        "punching",
        "running",
        "jumping once",
        "drinking with the left hand",
        "bowing",
        "throwing a ball",
        "shaking hands",
        "skipping rope",
        "picking something up with both hands",
        "dancing",
        "eating with the right hand",
        "waving",
        "catching a ball",
        "kicking a ball",
        "sitting down",
        "golfing (swinging a club)",
        "T-posing",
        "holding a baby",
    ]
    formatted_list_of_actions = "\n".join(f"- {action}" for action in list_of_actions)

    num_entries_on_page = 4
    num_actions = 4
    form_content_update = {
        "includeFormInResponse": False,
        "requests": [
            {
                "updateFormInfo": {
                    "info": {
                        "title": title,
                        "documentTitle": title,
                        "description": f"""This task is about question answering based on a short clip of a person performing four consecutive actions.

The depicted person performs exactly four (possibly equal) actions chosen from a pool of these actions (you don't need to remember them all):

{formatted_list_of_actions}

The four actions start every 4:00 seconds, resulting in:

- Seconds 00 to 04: activity #1
- Seconds 04 to 08: activity #2
- Seconds 08 to 12: activity #3
- Seconds 12 to 16: activity #4

The video will include the timestamps at the bottom.

Important note: If actions are repeated, they are considered to be one continuous action. For example, given the following sequence:

- Seconds 00 to 04: Bowing
- Seconds 04 to 08: Dancing
- Seconds 08 to 12: Dancing
- Seconds 12 to 16: Holding a baby

Then, for the question "How many times does the person dance?", the answer is "The person dances once.", not twice. If the question instead were "Was dancing performed the fewest times in the sequence?", the answer would similarly be "Yes"/"That is correct.", since all three distinct actions are performed once.

Please watch each video clip carefully and then answer the question based on your observations. If necessary, feel free to watch it again. It will loop until you submit your answer.

The answer should be given as a short but complete sentence with correct punctuation. There will only be 20 questions in total.""",
                    },
                    "updateMask": {
                        "paths": ["title", "documentTitle", "description"],
                    },
                },
            },
            *chain(
                [
                    {
                        "createItem": {
                            "item": {
                                "pageBreakItem": {},
                            },
                            "location": {"index": (num_entries_on_page * i) + 0},
                        },
                    },
                    # {
                    #     "createItem": {
                    #         "item": {
                    #             "title": "Reminder of the possible action types:",
                    #             "description": formatted_list_of_actions,
                    #             "textItem": {},
                    #         },
                    #         "location": {"index": (num_entries_on_page * i) + 1},
                    #     },
                    # },
                    {
                        "createItem": {
                            "item": {
                                "title": f"Question #{row['study_entry_id'] + 1}",
                                "imageItem": {
                                    "image": {"sourceUri": row["url"]},
                                },
                            },
                            "location": {"index": (num_entries_on_page * i) + 1},
                        },
                    },
                    # {
                    #                         "createItem": {
                    #                             "item": {
                    #                                 "title": "As a first step, please state the four actions you observed in the video. Please select '<not recognized>' if you cannot recognize it at all.",
                    #                                 "description": f"""All possible actions are listed here.
                    # {formatted_list_of_actions}""",
                    #                                 "textItem": {},
                    #                             },
                    #                             "location": {"index": (num_entries_on_page * i) + 2},
                    #                         },
                    #                     },
                    # *(
                    #     {
                    #         "createItem": {
                    #             "item": {
                    #                 "title": f"Action #{question_index}",
                    #                 "questionItem": {
                    #                     "question": {
                    #                         "required": True,
                    #                         # "questionId": f"{10*row['study_entry_id'] + question_index:08}",  # Doesn't work
                    #                         "choiceQuestion": {
                    #                             "type": "RADIO",
                    #                             "options": [
                    #                                 *({"value": action} for action in list_of_actions),
                    #                                 {"value": "?"},
                    #                             ],
                    #                             "shuffle": False,
                    #                         },
                    #                     }
                    #                 },
                    #             },
                    #             "location": {"index": (num_entries_on_page * i) + 2 + question_index},
                    #         },
                    #     }
                    #     for question_index in range(1, num_actions + 1)
                    # ),
                    {
                        "createItem": {
                            "item": {
                                "title": "Please first identify the four actions from in the video. Please select '<not recognized>' if you cannot recognize it at all.",
                                "questionGroupItem": {
                                    "questions": [
                                        {
                                            "required": True,
                                            "rowQuestion": {"title": f"Action #{question_index}"},
                                        }
                                        for question_index in range(1, num_actions + 1)
                                    ],
                                    "grid": {
                                        "columns": {
                                            "type": "RADIO",
                                            "options": [
                                                *({"value": action} for action in list_of_actions),
                                                {"value": "<not recognized>"},
                                            ],
                                        }
                                    },
                                },
                            },
                            "location": {"index": (num_entries_on_page * i) + 2},
                        }
                    },
                    {
                        "createItem": {
                            "item": {
                                "title": f"{row['question']} Please answer based on the actions you identified in FULL sentences.",
                                "questionItem": {
                                    "question": {
                                        "required": True,
                                        "questionId": f"{row['study_entry_id']:08}",
                                        # "choiceQuestion": {
                                        #     "type": "RADIO",
                                        #     "options": [
                                        #         # {"value": "Yes"},
                                        #         # {"value": "No"},
                                        #         {"value": "A"},
                                        #         {"value": "B"},
                                        #         {"value": "C"},
                                        #     ],
                                        #     "shuffle": False,
                                        # },
                                        "textQuestion": {"paragraph": True},
                                    }
                                },
                            },
                            "location": {"index": (num_entries_on_page * i) + 3},
                        },
                    },
                ]
                for i, row in form_data.iterrows()
            ),
            {
                "createItem": {
                    "item": {
                        "pageBreakItem": {},
                    },
                    "location": {"index": (num_entries_on_page * len(form_data) + 0)},
                },
            },
            {
                "createItem": {
                    "item": {
                        "title": "Prolific ID",
                        "description": """Please enter your Prolific ID below, so we can complete and approve your submission. This is very important! You prolific ID consits of numbers and letters and is 24 characters long.""",
                        "questionItem": {
                            "question": {
                                "questionId": "999999",
                                "required": True,
                                "textQuestion": {"paragraph": False},
                            }
                        },
                    },
                    "location": {"index": (num_entries_on_page * len(form_data) + 1)},
                },
            },
            {
                "createItem": {
                    "item": {
                        "pageBreakItem": {},
                    },
                    "location": {"index": (num_entries_on_page * len(form_data) + 2)},
                },
            },
            {
                "createItem": {
                    "item": {
                        "title": "Done!",
                        "description": f"""You have now completed the task. Please click on the link below to finalize the task:

[Back to Prolific]({APPROVAL_URL}).""",
                        "textItem": {},
                    },
                    "location": {"index": (num_entries_on_page * len(form_data) + 3)},
                },
            },
        ],
    }
    forms_service.forms().batchUpdate(formId=form_id, body=form_content_update).execute()

    # This does not work since the form is in the folder of the service account
    # drive_service.files().update(
    #     fileId=form_id,
    #     addParents="1yEu8OPKYru7c_4N0xcevr-8t-QQ1WUGT",
    #     # /QuAnTS/study/binary
    #     # , removeParents="root"
    #     fields="id, parents",
    # ).execute()

    # drive_service.permissions().create(
    #     fileId=form_id, body={"type": "user", "role": "writer", "emailAddress": PERSONAL_EMAIL}, fields="id"
    # ).execute()

    return form_id, result["responderUri"]


for block in tqdm(range(TOTAL_NUMBER // BLOCK_SIZE)):
    if any(res["block"] == block for res in all_results):
        print(f"Skipping block {block}")
        continue

    start_index = block * BLOCK_SIZE
    end_index = min(((block + 1) * BLOCK_SIZE), len(data))
    form_data = data[start_index:end_index].reset_index()

    form_id, responder_uri = create_form(block, form_data)
    print(f"Form #{block} created: {form_id}, {responder_uri}")

    all_results.append(
        {
            "block": block,
            "form_id": form_id,
            "responder_uri": responder_uri,
            "study_entry_ids": form_data["study_entry_id"].tolist(),
        }
    )
    as_csv += f"{block},\"{form_id}\",\"{responder_uri}\",\"{form_data['study_entry_id'].tolist()}\"\n"

    sleep(1)

all_results_df = pd.DataFrame(all_results)
all_results_df.to_csv("../../generated-dataset-30_000/human-study/open/study_forms_open.csv", index=False)

In [ ]:
print(as_csv)

In [ ]:
print(all_results_df["responder_uri"].to_csv(index=False, header=False))

In [ ]:
all_form_ids = pd.concat(
    [
        pd.read_csv("../../generated-dataset-30_000/human-study/binary/study_forms_binary.csv")["form_id"],
        pd.read_csv("../../generated-dataset-30_000/human-study/multi/study_forms_multi.csv")["form_id"],
        pd.read_csv("../../generated-dataset-30_000/human-study/open/study_forms_open.csv")["form_id"],
    ]
)
len(all_form_ids)

In [ ]:
drive_service = build("drive", "v3", credentials=credentials)

all_forms = (
    drive_service.files().list(q="mimeType='application/vnd.google-apps.form'", pageSize=1000).execute()
)
ids = [entry["id"] for entry in all_forms["files"] if entry["id"] not in set(all_form_ids)]
len(all_forms["files"]), len(ids)

In [ ]:
# for id in ids:
#     drive_service.files().delete(fileId=id).execute()